# Language Modelling

## General info

## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [ ]:
hangman('whatever', human, 8, True)

I will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

The first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set will contain 1000 word types, and the rest should be in the training set. 

In [3]:
from nltk.corpus import brown
import numpy as np
import re
# get unique word types ocurring in the Brown corpus selecting only words with alpahbetic characters - lowercase
brown_words = set([word.lower() for word in brown.words()])
words=list(set([word.lower() for word in brown_words if re.match(r'^[a-z]+$',word)]))
# split for training and testing sets
np.random.shuffle(words)
test_set=words[0:1000]
train_set=words[1000:len(words)]
print(len(test_set))
print(len(train_set))

1000
39234


The first *AI* attempt will be a trivial random method. For this I will implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. It randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game.

To measure the performance I will implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. To turn off the printouts for this, we use the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26` and get the average number of mistakes for the random AI.

In [4]:
# guessing method using the same input arguments as <human> method and returning a character
from string import ascii_lowercase

def trivial_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed] # from all letters in alphabet
    return np.random.choice(letter_available)
            
def evaluate_model(guesser,dataset,**guesser_args):
    verbose=False
    max_mistakes=26
    mistakes=0
    for word_test in dataset:   
        mistakes+=hangman(word_test, guesser, max_mistakes, verbose,**guesser_args)
      
    avg_mistakes=mistakes/len(dataset)
    return avg_mistakes
    


In [5]:
print(evaluate_model(trivial_guesser,test_set))

16.659


For the first real AI, I will train a *unigram* model over the training set.  This requires  to find the frequencies of characters over all training words. Using this model, I will have a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. 

In [6]:
# get the frequencies of characters over all training words
from collections import defaultdict
from collections import Counter


def get_unigram_counts(words):
    unigram_counts = Counter()
    
    # collect initial unigram statistics
    for word in words:
        for letter in word:
                unigram_counts[letter] += 1

                
    return unigram_counts


unigram_counts=get_unigram_counts(train_set)



To implement the Unigram model we need to calculate the probabilites for every character $c_i$:<br/>

$P_{unigram}(c_i)=\frac{count(c_i)}{M}$<br/>
where $M$ is the total number of characters

In [7]:
def unigram_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    total_counts = float(sum(unigram_counts.values()))
    unigram_probs={}
    vocab_size = len(unigram_counts)
    # calculate the probability for all letter available
    for letter in letter_available:
        unigram_probs[letter]=(unigram_counts[letter]+1)/(total_counts+vocab_size) #laplace smoothing
    
    # get the max probability 
    letter_choice=max(unigram_probs, key=unigram_probs.get)
    return letter_choice

        
# print the average number of mistakes the unigram method makes over the test set.
print(evaluate_model(unigram_guesser,test_set))

10.719


The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. We can incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word (Being careful of an unlikely situation that we encounter a word length that you didn't see in training). 

To implement the Unigram model conditioned we need to calculate the probabilites for every character $c_i$:<br/>

$P_{unigram}(c_i|n)=\frac{count(c_i,n)}{M}$<br/>
where $M$ is the total number of characters and $n$ is a fixed lenght for a word

In [8]:
# implement conditioning unigram model on the lengh of the word - different models according the lenght
# be careful with unseen lenghts in training
# create guessing function using the new model and print performance
def get_conditional_unigram_counts(words):    
    unigram_lenght_counts=defaultdict(Counter)
    total_counts=0.0
    
    # collect initial unigram statistics
    for word in words:
        for letter in word:
            unigram_lenght_counts[len(word)][letter] += 1
            
                
    return unigram_lenght_counts

unigram_lenght_counts=get_conditional_unigram_counts(train_set)

def unigram_conditional_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    total_counts = float(sum(unigram_lenght_counts[len(mask)].values()))
    unigram_probs={}
    vocab_size = len(unigram_lenght_counts)
    # calculate the probability for all letter available
    for letter in letter_available:
        # if the word len is not in the training set total_counts=0. 
        if (total_counts!=0):
            unigram_probs[letter]=(unigram_lenght_counts[len(mask)][letter]+1)/(total_counts+vocab_size)
        else:
            # out-of-vocabulary words = 1 / |V|
            vocab_size = len(unigram_lenght_counts)
            zerogram_prob = (1 / float(vocab_size)) 
            unigram_probs[letter]=zerogram_prob 
            
            
    # get the max probability 
    letter_choice=max(unigram_probs, key=unigram_probs.get)
    return letter_choice

In [9]:
# print the average number of mistakes the unigram method makes oert the test set.
print(evaluate_model(unigram_conditional_guesser,test_set))

10.735


Now I will be using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).
For this model, I use linear interpolation to smooth between the higher order and lower order models.

I will apply the language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, I sum over the probability distributions for each blank to find the expected count for each character type and select the character with the highest expected count.


In [10]:
import math

def convert_word(word,n):
    start=[]
    end=[]
    start_index=1
    
    # padding with sentinent symbols
        
    while start_index<n:
        start.append("<s"+str(start_index)+">")
        start_index+=1
    
    end_index=n
    while end_index>1:
        end.append("</s"+str(end_index-1)+">")
        end_index-=1
    
    return start + [l.lower() for l in word] + end

def get_ngram_counts(words,n):
    ngram_counts = defaultdict(Counter)
    
    # collect bigram counts
    for word in words:
        word = convert_word(word,n)
        if (n<=len(word)):
            n_grams=[word[i:i+n] for i in range(len(word)-(n-1))]
            for n_gram in n_grams:
                if (n==2):
                    ngram_counts[n_gram[0]][n_gram[1]]+=1
                elif (n==3):
                    ngram_counts[(n_gram[0],n_gram[1])][n_gram[2]]+=1
                elif (n==4):
                    ngram_counts[(n_gram[0],n_gram[1],n_gram[2])][n_gram[3]]+=1
                elif (n==5):
                    ngram_counts[(n_gram[0],n_gram[1],n_gram[2],n_gram[3])][n_gram[4]]+=1

    return ngram_counts





In [11]:
# train ngram where n = 2...5.
bigram_counts=get_ngram_counts(train_set,2)
trigram_counts=get_ngram_counts(train_set,3)
fourgram_counts=get_ngram_counts(train_set,4)
fivegram_counts=get_ngram_counts(train_set,5)

# dictionary for query in the guesser
ngram_counts = {1 : unigram_counts,
           2 : bigram_counts,
           3 : trigram_counts,
           4 : fourgram_counts,
           5 : fivegram_counts
    }

In [12]:
def get_ngram_prob(letter,context,n,lambdas):
    
    
    prob=0
    
    # create tuples to make a query with context
    if (n>1):
        if (len(context)>1):
            conditional=tuple(context)
        elif(len(context)==1):
            conditional=context[0]
        else:
            conditional=tuple() # no context at all
    
            

    if (n>4):
        
        # calculate the fivegram
        fivegram_count=ngram_counts[5][conditional][letter]*lambdas[5]
        fivegram_total_count=float(sum(ngram_counts[5][conditional].values()))
        if fivegram_total_count!=0:
            interp_prob_fivegram=fivegram_count/fivegram_total_count
        else:
            interp_prob_fivegram=0
            lambdas[4]+=lambdas[5] #if count is 0 I will give the lambda weight to the next ngram
            
        prob+=interp_prob_fivegram
        
    if (n>3):
        # calculate the fourgram
        fourgram_count=ngram_counts[4][conditional][letter]*lambdas[4]
        fourgram_total_count=float(sum(ngram_counts[4][conditional].values()))
        if fourgram_total_count!=0:
            interp_prob_fourgram=fourgram_count/fourgram_total_count
        else:
            interp_prob_fourgram=0
            lambdas[3]+=lambdas[4] #if count is 0 I will give the lambda weight to the next ngram
            
        prob+=interp_prob_fourgram
    
    if (n>2):
        # calculate the trigram
        trigram_count=ngram_counts[3][conditional][letter]*lambdas[3]
        trigram_total_count=float(sum(ngram_counts[3][conditional].values()))
        if trigram_total_count!=0:
            interp_prob_trigram=trigram_count/trigram_total_count
        else:
            interp_prob_trigram=0
            lambdas[2]+=lambdas[3] #if count is 0 I will give the lambda weight to the next ngram
        
        prob+=interp_prob_trigram
        
    if (n>1):
        # calculate the bigram
        bigram_count=ngram_counts[2][conditional][letter]*lambdas[2]
        bigram_total_count=float(sum(ngram_counts[2][conditional].values()))
        if bigram_total_count!=0:
            interp_prob_bigram=bigram_count/bigram_total_count
        else:
            interp_prob_bigram=0
            lambdas[1]+=lambdas[2] #if count is 0 I will give the lambda weight to the next ngram
        
        prob+=interp_prob_bigram
    
    if (n>0):
        unigram_count=ngram_counts[1][letter]*lambdas[1]
        unigram_total_count=float(sum(ngram_counts[1].values()))
        if unigram_total_count!=0:
            interp_prob_unigram=(unigram_count)/(unigram_total_count) # not smoothed with laplace like the unigram model       
        else:
            
            # out-of-vocabulary words = 1 / |V|
            lambdas[0]+=lambdas[1]
            vocab_size = len(unigram_total_count)
            interp_prob_unigram = (1 / float(vocab_size)) * lambdas[0]
        prob+=interp_prob_unigram
        
        
    return math.log(prob)
     

In [13]:
def ngram_guesser(mask, guessed, **kwargs):
    letter_available=[letter for letter in ascii_lowercase if letter not in guessed]
    blank_probs={}
    letter_probs={}
    probs=[]
    if kwargs.get('n'):       
        n=kwargs.get('n')
    else:
        n=1
        
    if kwargs.get('lambdas'):       
        lambdas=kwargs.get('lambdas')
    else:
        lambdas={1:1}
    word=convert_word(mask,2)
    letter_probs=defaultdict(float)
    for i in range(len(word)):
        if (word[i]=="_"):
            context=[]
            j=i-1
            context_len=1
            while word[j]!="_" and j>=0 and context_len<n:
                context.insert(0,word[j])
                j-=1
                context_len+=1
            
            for letter in letter_available:
                probability=get_ngram_prob(letter,context.copy(),n,lambdas.copy())
                letter_probs[letter]+=probability
                
              
    letter_choice=max(letter_probs, key=letter_probs.get)
    return letter_choice


In [14]:
lambdas_range=[0.98,0.95,0.8,0.5,0.25,0.001]

In [15]:

for bigram_lambda in lambdas_range:
    zerogram_lambda=0.01
    unigram_lambda = 1- zerogram_lambda - bigram_lambda
    lambdas_bigram = {0:zerogram_lambda,
           1 : unigram_lambda,
           2 : bigram_lambda
    }
    
    print('Lambdas (Sum='+str(sum(lambdas_bigram.values()))+'):',lambdas_bigram,'Avg Mistakes:',evaluate_model(ngram_guesser,test_set,n=2, lambdas=lambdas_bigram))

Lambdas (Sum=1.0): {0: 0.01, 1: 0.010000000000000009, 2: 0.98} Avg Mistakes: 8.783
Lambdas (Sum=1.0): {0: 0.01, 1: 0.040000000000000036, 2: 0.95} Avg Mistakes: 8.828
Lambdas (Sum=1.0): {0: 0.01, 1: 0.18999999999999995, 2: 0.8} Avg Mistakes: 9.018
Lambdas (Sum=1.0): {0: 0.01, 1: 0.49, 2: 0.5} Avg Mistakes: 9.476
Lambdas (Sum=1.0): {0: 0.01, 1: 0.74, 2: 0.25} Avg Mistakes: 9.949
Lambdas (Sum=1.0): {0: 0.01, 1: 0.989, 2: 0.001} Avg Mistakes: 10.719


In [16]:
import random

for trigram_lambda in lambdas_range:
    zerogram_lambda=0.01
    bigram_lambda = random.uniform(0.8, 1)*(1-zerogram_lambda-trigram_lambda)
    unigram_lambda=1-trigram_lambda-bigram_lambda-zerogram_lambda
    lambdas_trigram = {0:zerogram_lambda,
           1 : unigram_lambda,
           2 : bigram_lambda,
            3: trigram_lambda
    }
    print('Lambdas (Sum='+str(sum(lambdas_trigram.values()))+'):',lambdas_trigram,'Avg Mistakes:',evaluate_model(ngram_guesser,test_set,n=3, lambdas=lambdas_trigram))

Lambdas (Sum=1.0): {0: 0.01, 1: 0.00156637158891027, 2: 0.008433628411089748, 3: 0.98} Avg Mistakes: 8.74
Lambdas (Sum=1.0): {0: 0.01, 1: 0.0014643439003896534, 2: 0.03853565609961039, 3: 0.95} Avg Mistakes: 8.758
Lambdas (Sum=1.0): {0: 0.01, 1: 0.0012604792228405413, 2: 0.18873952077715941, 3: 0.8} Avg Mistakes: 8.836
Lambdas (Sum=1.0): {0: 0.01, 1: 0.08731855318421734, 2: 0.40268144681578266, 3: 0.5} Avg Mistakes: 9.146
Lambdas (Sum=1.0): {0: 0.01, 1: 0.04754127009828391, 2: 0.6924587299017161, 3: 0.25} Avg Mistakes: 9.48
Lambdas (Sum=1.0): {0: 0.01, 1: 0.08243640015949828, 2: 0.9065635998405017, 3: 0.001} Avg Mistakes: 10.353


In [17]:
import random

for fourgram_lambda in lambdas_range:
    zerogram_lambda=0.01
    trigram_lambda=random.uniform(0, 1)*(1-zerogram_lambda-fourgram_lambda)
    bigram_lambda = random.uniform(0, 1)*(1-zerogram_lambda-fourgram_lambda-trigram_lambda)
    unigram_lambda=1-fourgram_lambda-trigram_lambda-bigram_lambda-zerogram_lambda
    lambdas_fourgram = {0:zerogram_lambda,
        1 : unigram_lambda,
        2 : bigram_lambda,
        3 : trigram_lambda,
        4 : fourgram_lambda
    }
    print('Lambdas (Sum='+str(sum(lambdas_fourgram.values()))+'):',lambdas_fourgram,'Avg Mistakes:',evaluate_model(ngram_guesser,test_set,n=4, lambdas=lambdas_fourgram))

Lambdas (Sum=1.0): {0: 0.01, 1: 0.0009222287952319712, 2: 0.007705275923174914, 3: 0.0013724952815931336, 4: 0.98} Avg Mistakes: 8.831
Lambdas (Sum=1.0): {0: 0.01, 1: 0.008837736303773177, 2: 0.0007930191877357932, 3: 0.030369244508491072, 4: 0.95} Avg Mistakes: 8.845
Lambdas (Sum=1.0): {0: 0.01, 1: 0.038698147023799, 2: 0.09542031171547886, 3: 0.05588154126072208, 4: 0.8} Avg Mistakes: 8.857
Lambdas (Sum=1.0): {0: 0.01, 1: 0.25595042574021354, 2: 0.1321738511705646, 3: 0.10187572308922185, 4: 0.5} Avg Mistakes: 9.076
Lambdas (Sum=1.0): {0: 0.01, 1: 0.15566851954323147, 2: 0.007145582416609344, 3: 0.5771858980401592, 4: 0.25} Avg Mistakes: 9.272
Lambdas (Sum=1.0): {0: 0.01, 1: 0.22209746238399786, 2: 0.7329895426513903, 3: 0.03391299496461189, 4: 0.001} Avg Mistakes: 10.373


In [18]:
import random

for fivegram_lambda in lambdas_range:
    zerogram_lambda=0.01
    fourgram_lambda=random.uniform(0, 1)*(1-zerogram_lambda-fivegram_lambda)
    trigram_lambda=random.uniform(0, 1)*(1-zerogram_lambda-fivegram_lambda-fourgram_lambda)
    bigram_lambda = random.uniform(0, 1)*(1-zerogram_lambda-fivegram_lambda-fourgram_lambda-trigram_lambda)
    unigram_lambda=1-fivegram_lambda-fourgram_lambda-trigram_lambda-bigram_lambda-zerogram_lambda
    lambdas_fivegram = {0:zerogram_lambda,
        1 : unigram_lambda,
        2 : bigram_lambda,
        3 : trigram_lambda,
        4 : fourgram_lambda,
        5 : fivegram_lambda
    }
    print('Lambdas (Sum='+str(sum(lambdas_fivegram.values()))+'):',lambdas_fivegram,'Avg Mistakes:',evaluate_model(ngram_guesser,test_set,n=5, lambdas=lambdas_fivegram))

Lambdas (Sum=1.0): {0: 0.01, 1: 0.0012130405635881018, 2: 0.0057118991780785704, 3: 0.000931490429769466, 4: 0.0021435698285638778, 5: 0.98} Avg Mistakes: 9.313
Lambdas (Sum=1.0): {0: 0.01, 1: 0.0003260938048536468, 2: 0.002436892640600408, 3: 0.009257147987483788, 4: 0.027979865567062202, 5: 0.95} Avg Mistakes: 9.314
Lambdas (Sum=1.0): {0: 0.01, 1: 0.012058112051390261, 2: 0.013026680483510241, 3: 0.06589560494357394, 4: 0.09901960252152552, 5: 0.8} Avg Mistakes: 9.294
Lambdas (Sum=1.0): {0: 0.01, 1: 0.02670378183883581, 2: 0.02643966681837347, 3: 0.369402593708379, 4: 0.06745395763441168, 5: 0.5} Avg Mistakes: 9.328
Lambdas (Sum=1.0): {0: 0.01, 1: 0.04082198138445369, 2: 0.0006568775816813249, 3: 0.3265139605867537, 4: 0.3720071804471113, 5: 0.25} Avg Mistakes: 9.386
Lambdas (Sum=0.9999999999999999): {0: 0.01, 1: 0.3790804804892498, 2: 0.17615481637541833, 3: 0.0953211335136957, 4: 0.3384435696216362, 5: 0.001} Avg Mistakes: 10.281


There is a significant improvement. The trigram is apparently the best model according to results. Additionally, the lambda values for smoothing seems to perform better when the weight is in favor for the highest model.